#### Física de la Radioterapia. Master de Física Biomédica. Universidad Complutense de Madrid
# Calibración del acelerador lineal
## Haces de fotones medidos en condiciones de referencia
-----

**Objetivos**: 
Determinar la dosis absorbida producida por un campo de radiación en condiciones estándar de calibración utilizando una cámara de ionización tipo Farmer y aplicando el [protocolo de dosimetría de la IAEA TRS398](https://www-pub.iaea.org/MTCD/publications/PDF/TRS_398s_Web.pdf). Se recomienda leer al menos el epígrafe **3. FORMALISMO BASADO EN** $N_{D,w}$.

**Antecendentes**: De acuerdo al TRS398 la magnitud $D_{w, Q}$, dosis en agua para la calidad de la radiación $Q$, se puede determinar mediante la expresión

\begin{equation}
    D_{w, Q} = M_Q\;N_{D, w, Q_0}\;k_{Q, Q_0}\;k_{sat}\;k_{pol}.
\end{equation}

donde:
- $M_Q$ es la lectura neta del electrómetro expresado en condiciones normales
- $N_{D, w, Q_0}$ es el valor del factor de calibración de la cámara en condiciones de referencia
- $k_{Q, Q_0}$ es el factor de corrección por la calidad del haz del factor de calibración
- $k_{sat}$ es el factor de corrección por saturación de la cámara
- $k_{pol}$ es factor de corrección por dependencia de la lectura de la cámara con la polarización de sus electrodos

**Datos**:
Los datos de la cámara empleada están recogidos en su [certificado de calibración](../docs/CamaraFarmer10820.pdf)

Previamente se ha determinado que para esta cámara los factores de corrección por saturación $k_{sat}$ y por polarización $k_{pol}$ son ambos igual a uno en las condiciones de medida consideradas.

La energía del haz empleado corresponde a una razón TPR${}_{20, 10} = 0.694$.

Para recolectar la carga producida en la cámara se utiliza un electrómetro con una resolución de $0.001$ nC.

## Cuestión
1. Obtener de los datos del TRS398 el factor de corrección de la calibración de la cámara por la energía para el haz utilizado.

**Ayuda**: Consultar el CUADRO 14 página 85.


## Módulo `simcaliblinac`
Este cuaderno simula las condiciones variables de la medida mediante generadores aleatorios que siguen distribuciones realistas. Para realizar el ejercicio cada alumno generará y recogerá tanto las medidas como las condiciones en las que las ha realizado.

El módulo `simcaliblinac` contiene las funciones para realizar esta simulación.

En la siguiente celda importamos el módulo

In [1]:
import simcaliblinac as scl

Tomar los valores de la presión atmosférica indicada por el barómetro del Servicio y de la temperatura medida por la sonda del termómetro que introducimos en el agua. 

Las unidades de las medidas son presión en hPa y temperatura en grados centígrados.

In [2]:
presion, temperatura = scl.presion_temperatura_medida()
presion, temperatura

(array([947.25546126]), array([[20.37662419]]))

El módulo `simcaliblinac` ofrece una función para calcular la corrección por presión y temperatura $\phi$pT

In [3]:
ϕpT = scl.ϕpTf(presion, temperatura)
ϕpT

array([[1.07077874]])

Una vez montados los equipos, fijadas las condiciones experimentales y con todas las conexiones realizadas, comenzamos por medir la lecutra del electrómetro cuando no hay radiación. En Radioterapia a este valor lo denominamos *corriente de fugas*. Es el equivalente a una lectura de un valor de fondo pero de origen diferente.

Las fugas son de naturaleza aleatoria. Su valor medio se estima integrando durante un tiempo dado suficiente como para promediar variaciones instantáneas. 

El módulo `simcaliblinac` expone un método para obtener una muestra de las fugas recolectadas durante un tiempo dado

In [4]:
fugas = scl.fugas(minutos=2)
fugas # en nanoCulombios

array([[0.16707123]])

Para hacer las medidas en el acelerador tenemos que fijar un número de unidades monitor nominales.

In [5]:
UM_nominales = 200

El módulo `simcaliblinac` expone un método para obtener una muestra de **n** medidas con las unidades monitor fijadas

In [6]:
lecturas = scl.lectura_medida_f(UM=UM_nominales, n=3)
lecturas # en nanoCulombios

array([[35.54802083, 35.55559551, 35.51982875]])

Internamente el módulo nos permite saber cual es el valor esperado actual de la lectura

In [7]:
scl._lectura_actual_f(UM=UM_nominales)

35.54076016016185

El módulo simcaliblinac expone métodos para calular la dosis para el número de unidades monitor fijado y el output del acelerador (dosis en Gy por cada 100 UM), una vez que se han determinado las constantes de la cámara de acuerdo al TRS398.

In [8]:
# Constantes de la cámara. El alumno las tiene que obtener del TRS398
NDwQ0 = 0.05396
kQ = 0.989

Dosis medidas

In [9]:
D_meds = scl.dosis_f(lecturas, UM=UM_nominales, ϕpT=ϕpT, fugas=fugas/2, NDwQ0=NDwQ0, kQ=kQ)
D_meds

array([[2.02975246, 2.0301853 , 2.02814146]])

Output medidos

In [10]:
output_meds = scl.output_f(lecturas, UM=UM_nominales, ϕpT=ϕpT, fugas=fugas/2, NDwQ0=NDwQ0, kQ=kQ)
output_meds

array([[1.01487623, 1.01509265, 1.01407073]])

## Cuestión
2. Determinar la dosis absorbida suministrada por el acelerador
1. Hacer una estimación de la incertidumbre en la medida de la dosis absorbida

## Linealidad

El número de UM empleado para los tratamientos se encuentra en un amplio rango que va desde unas pocas UM hasta varios miles. Para comprobar el funcionamiento de las cámaras en ese rango se realizan medidas de linealidad, en las que se comprueba la proporcinalidad del cambio de la respuesta con el cambio de UM.

In [11]:
import numpy as np
import matplotlib.pyplot as plt

In [12]:
UMs = np.array([100, 200, 500, 1000, 1500, 3000])

In [13]:
UMs

array([ 100,  200,  500, 1000, 1500, 3000])

In [19]:
scl.lectura_medida_f(UMs, n=2, trials=3)

array([[[ 17.77225752,  17.79112825],
        [ 17.77472205,  17.73808454],
        [ 17.75364101,  17.77418945]],

       [[ 35.54641811,  35.53081603],
        [ 35.60788923,  35.5436979 ],
        [ 35.5257728 ,  35.55877813]],

       [[ 89.06975725,  88.81749047],
        [ 88.77597751,  88.84027939],
        [ 88.99864828,  88.89466504]],

       [[177.97214027, 177.4796838 ],
        [177.73795146, 177.6323499 ],
        [177.94424204, 177.28296276]],

       [[266.97075033, 266.55063572],
        [266.21184014, 266.52724616],
        [266.86944968, 267.16228847]],

       [[532.18754442, 532.19787165],
        [532.81054184, 533.95400527],
        [532.51566483, 533.58854514]]])